In [1]:
import pandas as pd
import numpy as np
import warnings
import pandas as pd
import json
import numpy as np
from tqdm import tqdm
from urllib.request import urlopen
warnings.filterwarnings('ignore')
from deep_translator import LingueeTranslator, GoogleTranslator
import re
import multiprocessing

In [24]:
# get attributes:
def get_startYear(dateCreated):
    if dateCreated:
        return str(dateCreated[0]['startYear'])
    else:
        return ''

def get_endYear(dateCreated):
    if dateCreated:
        return str(dateCreated[0]['endYear'])
    else:
        return ''

def get_image(image):
    if image:
        return image[0]['urlImage']
    else:
        return ''

def get_dimension(dimension):
    if dimension:
        dim = dimension[0]['type'] + ': ' + dimension[0]['value'] + dimension[0]['unit']
        return dim
    else:
        return ''

def get_materials(materialsAndTechniques):
    return materialsAndTechniques

def get_place(placeOfCreation):
    return placeOfCreation

def get_description(description):
    return description

def get_inscription(inscription):
    return inscription

def get_category(index):
    if index:
        if 'category' in index:
            category = " ".join([i['value'] for i in index['category']])
            return category
    else:
        return ''

def get_url(url):
    return url

In [22]:
def get_data(arkId):
    try:
        data = np.empty([1, 11], dtype=object)
        root = 'https://collections.louvre.fr/ark:/53355/'
        json_url = root + arkId + '.json'
        response = urlopen(json_url)
        data_json = json.loads(response.read())

        # get datas:
        data[0,0] = arkId
        data[0,1] = get_startYear(data_json['dateCreated'])
        data[0,2] = get_endYear(data_json['dateCreated'])
        data[0,3] = get_category(data_json['index'])
        data[0,4] = get_place(data_json['placeOfCreation'])
        data[0,5] = get_dimension(data_json['dimension'])
        data[0,6] = get_materials(data_json['materialsAndTechniques'])
        data[0,7] = get_description(data_json['description'])
        data[0,8] = get_inscription(data_json['inscriptions'])
        data[0,9] = get_image(data_json['image'])
        data[0,10] = get_url(data_json['url'])
        response.close()
        return data
    except:
        print('Error')
        pass

In [86]:
data = np.empty([1, 11], dtype=object)
def get_df(file_path):
    df = pd.read_csv(file_path)
    features = ['ARK', 'startYear', 'endYear', 'category', 'place', 'dimension', 'materials', 'description', 'inscriptions', 'image', 'url']
    ARK = df['ARK'].tolist()
    data = np.empty([1, 11], dtype=object)
    for arkId in tqdm(ARK):
        data_ark = get_data(arkId)
        data = np.append(data, data_ark, axis=0)
    df1 = pd.DataFrame(data, columns = features)
    df = pd.merge(df, df1, how='inner', on='ARK')
    return df

In [1]:
data = get_df('paintings.csv')

In [102]:
# data.to_csv('louvre.csv')

In [ ]:
## Translation:

In [ ]:
def remove_punctuation(s):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~='''
    s = ''.join([i.lower() if not i in punc else ' ' for i in s])
    s = ' '.join([i for i in s.split('\n') if i != ' '])
    s = ' '.join([i for i in s.split('\r') if i != ' '])
    s = ' '.join([i for i in s.split('\t') if i != ' '])
    return s

In [3]:
# Phrase to phrase translation:
for i in ['title', 'category', 'place', 'materials']:
    feat_list = data['title'].tolist()
    english = []
    for i in tqdm(feat_list):
        english.append(translated(i, 'french', 'english'))
    name = i+'_en'
    data[name] = title_english

# word by word translation of description:
description = data['description'].tolist()
description_words = []
for i in description:
    description_words += [j.strip() for j in i.split(' ')]
description_words = list(set(description_words))
len(description_words)
description_word_en_1 = []
for i in tqdm(description_words):
    if type(i) == float:
        description_word_en_1.append('')
    elif i.isnumeric():
        description_word_en_1.append(i)
    else:
        description_word_en_1.append(translated(i, 'french', 'english'))
        
description_word_dict = dict(zip(description_words, description_word_en_1))
description = data['description'].tolist()
description_english = []
for i in tqdm(description):
    if type(i) == float:
        description_english.append('')
    else:
        d = ' '.join([description_word_dict[j].lower() for j in i.split(' ') if j.isalpha()])
        description_english.append(d)
data['description_en'] = description_english